# Predicting future states with Markov Chains
We have a set of sequences. We want to use Markov Chains to obtain predictions on future states. Also we want this representation to be of order $>1$.  

In [15]:
# load all dependencies
from support import *
# load the data
data = pd.read_csv("../data/sequences.csv", sep=";")[["element", "exit_codes"]]

In [16]:
# load data
print(f"Dataset loaded: {data.shape[0]} rows x {data.shape[1]} columns")
# let's peek into the dataframe...
data.head()

Dataset loaded: 61 rows x 2 columns


,element,exit_codes
0,6114|421|2814|1478|2040|3563|3622|1850|1032|20...,489|397|397|397|218|580|465|397|218|397|397|77...
1,6114|421|2814|1478|2040|3563|3622|1850|1032|20...,489|397|397|397|218|580|465|397|218|397|397|77...
2,6114|421|2814|1478|2040|3563|3622|1850|1032|20...,489|397|397|397|218|580|465|397|218|397|397|77...
3,6114|421|2814|1478|2040|3563|3622|1850|1032|20...,489|397|397|397|218|580|465|397|218|397|397|77...
4,6114|421|2814|1478|2040|3563|3622|1850|1032|20...,489|397|397|397|218|580|465|397|218|397|397|77...


### Data format
Each record consists of two sequences (events and outcomes). We need to associate each event (`element`) to its outcome (`exit_code`).

We need to determine the set of ALL possible events and outcomes. To do so we'll generate a hash map to a numeric range.

E.g.
```python
hashmap = {0: state0, 1: state1, 2: state2, ...}
```
for all states and outcomes. We sill end up with a dictionary of size `len(set(events)) + len(set(outcomes))`.

In [17]:
# get ALL the elements (pipe separated)
all_codes = set(
    itertools.chain(
        *data['element'].apply(lambda x: str(x).split("|")).values
    )
)
print(f"{len(all_codes)} unique elements found in dataset")

# build a label map
# this generates a new hash map with the range of the unique labels as key
codes_map = {
    item: n for n, item in enumerate(
        sorted(map(int, all_codes))
    )
}
# get the new labels and joins them in a string like at the beginning
data['new_elements'] = data['element'].apply(
    lambda x: "|".join(
        str(codes_map.get(int(i), 'N/A')) for i in str(x).split("|")
    )
)

# same thing on exit states
all_codes = set(
    itertools.chain(*data['exit_codes'].apply(lambda x: str(x).split("|")).values)
)
print(f"{len(all_codes)} unique exit_codes found in dataset")
_codes_map = {
    item: n for n, item in enumerate(
        sorted(map(int, all_codes)), start=len(codes_map)
    )
}
data['new_exits'] = data['exit_codes'].apply(
    lambda x: "|".join(
        str(_codes_map.get(int(i), 'N/A')) for i in str(x).split("|")
    )
)

print(f"\n{max(_codes_map.values()) + 1} possible states single states")

288 unique elements found in dataset
97 unique exit_codes found in dataset

385 possible states single states


In [18]:
# we don't need these anymore...
data.drop(["element", "exit_codes"], axis=1, inplace=True)
data.head()

,new_elements,new_exits
0,287|25|91|49|82|108|112|72|42|79|50|1|12|256|4...,371|361|361|361|325|384|366|361|325|361|361|30...
1,287|25|91|49|82|108|112|72|42|79|50|1|12|256|4...,371|361|361|361|325|384|366|361|325|361|361|30...
2,287|25|91|49|82|108|112|72|42|79|50|1|12|256|4...,371|361|361|361|325|384|366|361|325|361|361|30...
3,287|25|91|49|82|108|112|72|42|79|50|1|12|256|4...,371|361|361|361|325|384|366|361|325|361|361|30...
4,287|25|91|49|82|108|112|72|42|79|50|1|12|256|4...,371|361|361|361|325|384|366|361|325|361|361|30...


Let's now associate each element to its exit code in the sequence...

In [19]:
# this mixes and matches items and exit codes:
# a|b|c $ 1|2|3 => a|1|b|2|c|3
pair_sequences = make_sequences(data)

Trust me... it works...

## Let's do some coding!
Since we are interested in the pairs (event, outcome), we'll use a $2^{nd}$ order MC to generate our model.  
But first we need to implement the general MC.

## Implementation
How can we implement the representation of these states into a Markov Chain?  

First of all, we want an extensible object (it doesn't strictly **NEED** to be, but this will be helpful in keeping things tidy).

**CAVEAT**: I'm going to show the method implementation separately for clarity: they should ALL be in the class for it to work properly!!

**CAVEAT 2**: I'm omitting a lot of stuff from the code below and leaving a few imprecisions (error handling, logging, docstrings, transposing, ...) for clarity... I *DO NOT* advise you use this code directly: I'm pretty sure you'd run into some weird (and not weird) problems...

### Let's implement the class

Let's start simple...
```python
class MarkovChain(object):
    pass
```

#### Initialisation

ok... now we want to pre-allocate the transition matrix. To do it, we need to know the total number of states and the order of the MC.  
So we need to set:
1. order of the MC
2. total number of states

```python
    def __init__(self, n_states, order=1):
        self.number_of_states = n_states
        self.order = order
        
        # calculate all possible state combinations
        self.possible_states = {
            j: i for i, j in
            enumerate(itertools.product(range(n_states), repeat=order))
        }

        # allocate transition matrix
        self.transition_matrix = sparse.dok_matrix((
            (len(self.possible_states), len(self.possible_states))
        ), dtype=np.float64)
```


Good. At this point we have the initialisations and the objects we need to start working. We'll have to be careful when working with a lot of states or a very high order as the size of the representation of the transition matrix will grow as $O(n\_states^2)$. This is actually only a big problem if we want to have dense representations... which we don't...


#### Transition matrix update

We now have a basic implementation of and initialisation of the `transition_matrix` and `possible_states`. The transition matrizx has all transitions initialised as zeros, we now need a way of updating its elements when we see a transition happen:

```python
    def update_transition_matrix(self, states_sequence):

        visited_states = [
            states_sequence[i: i + self.order]
            for i in range(len(states_sequence) - self.order + 1)
        ]

        for state_index, i in enumerate(visited_states):
            self.transition_matrix[
                self.possible_states[tuple(i)],
                self.possible_states[tuple(visited_states[
                    state_index + self.order
                ])]
            ] += 1
```
This function updates the transition matrix for a single sequence of states. Something is missing though...

### Normalisation

The transition matrix is a matrix of probabilities, therefore **each row MUST sum to 1**.  

We need to implement this normalisation:

```python
    def normalise_transitions(self):
        self.transition_matrix = preprocessing.normalize(
            self.transition_matrix, norm="l1"
        )
```

### Fitting sequences of sequences

We now have most of the bits and pieces together: we can add a utility function to train the model on sequences of sequences (we don't want to explicitly call `update_transition_matrix`, do we? ;) ).

```python

    def fit(self, state_sequences):
        for index, sequence in enumerate(state_sequences):
            self.update_transition_matrix(sequence, normalize=False)
        self.normalize_transitions()
```

ok... that was easy...

#### Implementation of `predict_state`

We can implement this easily: we just need to take the matrix power of T to the power of the number of steps we want. For a single step, we'll be using the raw transition matrix:

```python
    def predict_state(self, current_state, num_steps=1):
        _next_state = sparse.csr_matrix(current_state).dot(
            np.power(self.transition_matrix, num_steps)
        )

        return _next_state
```

## Let's see it in action now!!
Ok fine, we've seen what the implementation looks like, let's see what it can do now!  
As mentioned, the implementation seeen above is not complete and it does not implement any form of error hanlding... Luckily for us, a more complete implementation exists!

```bash
# install the High Order Markov chains module
$ pip install --upgrade HOMarkov
```

In [20]:
import markov

In [21]:
number_of_states = max(_codes_map.values()) + 1 # zero based :)
mc = markov.MarkovChain(number_of_states, order=2)

In [22]:
%%time
# ~ 225 ms with small dataset (61), ~ 30s with big dataset (3.8K)
mc.fit(list(pair_sequences))

CPU times: user 253 ms, sys: 3.4 ms, total: 257 ms
Wall time: 255 ms


In [23]:
print(f"Number of non-zero transitions: {mc.transition_matrix.count_nonzero()}")
print(f"Number of total transitions: {mc.transition_matrix.shape[0]**2}")
print(f"Percentage of non-zero transitions: {mc.transition_matrix.count_nonzero() / mc.transition_matrix.shape[0]**2:%}")

Number of non-zero transitions: 886
Number of total transitions: 21970650625
Percentage of non-zero transitions: 0.000004%


As expected, this is a **VERY** sparse matrix!!  
We can now get all non-zero elements in the transition matrix and represent them with their states:

## Visualise the future
Now we have all the necessary building blocks to build a small dashboard to let us peek into the future.  
We'll be using `networkx` to generate a time graph of the state evolution.

In [24]:
# get coordinates of non-zero elements in transition matrix
non_zero_indices = list(zip(*mc.transition_matrix.nonzero()))
state_lookup = mc.possible_states_lookup()
state_transitions = [(state_lookup.get(i), state_lookup.get(j)) for i, j in non_zero_indices]
possible_start_states = set(i for i, _ in state_transitions if i[0] < 288)

In [25]:
@interact(
    initial_state=map(str, sorted(possible_start_states)),
    steps=IntSlider(1, 1, 5),
    threshold=IntSlider(0, 0, 100, 5),
    actual_weight=True
)
def draw_future(initial_state, steps, threshold, actual_weight):
    
    if not actual_weight:
        print("Using relative weight")
        
    state_index = (mc.possible_states.get(eval(initial_state)))
    state_representation = np.zeros(len(mc.possible_states))
    state_representation[state_index] = 1
    
    try:
        states = mc.evolve_states(state_representation, num_steps=steps, threshold=threshold*0.01)
    except TypeError as exc:  # no states above threshold
        print("No states above current threshold")
    else:
        pos = build_pos(states)  
        G = mc.generate_graph(states, actual=actual_weight)
        edge_labels = reset_edge_labels(G)
        labels = build_node_labels(G, state_lookup)

        nx.draw_networkx_edge_labels(G, pos, font_size=15, font_weight="bold", edge_labels=edge_labels, alpha=0.7)
        nx.draw_networkx(
            G,
            pos,
            width=[attr["weight"] * 5  for i, j, attr in G.edges(data=True)],
            edge_color=[attr["weight"] * 5 + 1  for i, j, attr in G.edges(data=True)],
            edge_cmap=optum_cmap_simple
        )
        label_offset = max([abs(i[1]) for i in pos.values()] + [0.05])
        nx.draw_networkx_labels(
            G,
            {k: (v[0], v[1] + 0.05 * label_offset) for k, v in pos.items()},
            labels,
            font_weight="bold",
            font_size=15
        )
        plt.axis("off")
        plt.show()
        
        # 51, 361
        # 58, 361
        # 65, 361  **
        # 76, 372
        # 216, 361
        # 261, 361

#### And there we go...
<center>
<img src="https://images.hellogiggles.com/uploads/2016/06/21071232/chris-pratt1.jpg" style="width:580px;height:350px">
</center>
